# Trading

- https://github.com/kelvinau/crypto-arbitrage : Arbitrage, not pip installable, not many exchanges
- https://github.com/ScottFreeLLC/AlphaPy : Machine learning framework for prediction
- https://github.com/ssatia/cointreau : LSTM trading model, not pip instalable, use for a workflow example
- https://github.com/Endogen/Telegram-Kraken-Bot : Interactive trading bot through Telegram
- https://github.com/bmoscon/cryptofeed : Normalizes prices across exchanges but doesn't use ccxt.
- https://github.com/samre12/gym-cryptotrading : Uses AI gym and reinforcement learning
- https://github.com/meister245/CryptoLunch : Find all arbitrage oppurtunities from coinmarketcap listed exchanges for all coins
- https://github.com/owocki/pytrader : Machine learning trading platform


## Interesting

- https://github.com/gcarq/freqtrade : Telegram trading bot
- https://github.com/ccxt/ccxt : Exchange API Library
- https://github.com/Denton24646/Titan : Trading Framework but not pip installable. Good wiki on structure: https://github.com/Denton24646/Titan/wiki Scavenge?
- https://github.com/CryptoSignal/crypto-signal : Platform for signals, again not pip installable. Uses ccxt, plugs into telegram and others. Scavenge?
- https://github.com/ScottFreeLLC/AlphaPy : Machine learning framework for prediction
- https://github.com/blue-yonder/tsfresh : Automatic time series feature extraction


## Looking for

- Speech recognition type graph analysis


# Trading View Algos

- Open Close Cross Strategy

```
//@version=2

strategy(title = "Open Close Cross Strategy", shorttitle = "OCC Strategy", overlay = true, pyramiding = 0, default_qty_type = strategy.percent_of_equity, default_qty_value = 10)

// Revision:        1
// Author:          @JayRogers
//
// Description:
//  - Strategy based around Open-Close Crossovers.
// Setup:
//  - I have generally found that setting the strategy resolution to 3-4x that of the chart you are viewing
//    tends to yield the best results, regardless of which MA option you may choose (if any)
//  - Don't aim for perfection. Just aim to get a reasonably snug fit with the O-C band, with good runs of
//    green and red.
//  - Option to either use basic open and close series data, or pick your poison with a wide array of MA types.
//  - Optional trailing stop for damage mitigation if desired (can be toggled on/off)
//  - Positions get taken automagically following a crossover - which is why it's better to set the resolution
//    of the script greater than that of your chart, so that the trades get taken sooner rather than later.
//  - If you make use of the trailing stops, be sure to take your time tweaking the values. Cutting it too fine
//    will cost you profits but keep you safer, while letting them loose could lead to more drawdown than you
//    can handle.

// === INPUTS ===
useRes      = input(defval = true, title = "Use Alternate Resolution? ( recommended )")
stratRes    = input(defval = "120", title = "Set Resolution ( should not be lower than chart )", type = resolution)
useMA       = input(defval = true, title = "Use MA? ( otherwise use simple Open/Close data )")
basisType   = input(defval = "DEMA", title = "MA Type: SMA, EMA, DEMA, TEMA, WMA, VWMA, SMMA, HullMA, LSMA, ALMA ( case sensitive )", type = string)
basisLen    = input(defval = 14, title = "MA Period", minval = 1)
offsetSigma = input(defval = 6, title = "Offset for LSMA / Sigma for ALMA", minval = 0)
offsetALMA  = input(defval = 0.85, title = "Offset for ALMA", minval = 0, step = 0.01)
useStop     = input(defval = true, title = "Use Trailing Stop?")
slPoints    = input(defval = 200, title = "Stop Loss Trail Points", minval = 1)
slOffset    = input(defval = 400, title = "Stop Loss Trail Offset", minval = 1)
// === /INPUTS ===

// === BASE FUNCTIONS ===
// Returns MA input selection variant, default to SMA if blank or typo.
variant(type, src, len, offSig, offALMA) =>
    v1 = sma(src, len)                                                  // Simple
    v2 = ema(src, len)                                                  // Exponential
    v3 = 2 * v2 - ema(v2, len)                                          // Double Exponential
    v4 = 3 * (v2 - ema(v2, len)) + ema(ema(v2, len), len)               // Triple Exponential
    v5 = wma(src, len)                                                  // Weighted
    v6 = vwma(src, len)                                                 // Volume Weighted
    v7 = na(v5[1]) ? sma(src, len) : (v5[1] * (len - 1) + src) / len    // Smoothed
    v8 = wma(2 * wma(src, len / 2) - wma(src, len), round(sqrt(len)))   // Hull
    v9 = linreg(src, len, offSig)                                       // Least Squares
    v10 = alma(src, len, offALMA, offSig)                               // Arnaud Legoux
    type=="EMA"?v2 : type=="DEMA"?v3 : type=="TEMA"?v4 : type=="WMA"?v5 : type=="VWMA"?v6 : type=="SMMA"?v7 : type=="HullMA"?v8 : type=="LSMA"?v9 : type=="ALMA"?v10 : v1
// security wrapper for repeat calls
reso(exp, use, res) => use ? security(tickerid, res, exp) : exp
// === /BASE FUNCTIONS ===

// === SERIES SETUP ===
// open/close
closeSeries = useMA ? reso(variant(basisType, close, basisLen, offsetSigma, offsetALMA), useRes, stratRes) : reso(close, useRes, stratRes)
openSeries  = useMA ? reso(variant(basisType, open, basisLen, offsetSigma, offsetALMA), useRes, stratRes) : reso(open, useRes, stratRes)
trendState  = closeSeries > openSeries ? true : closeSeries < openSeries ? false : trendState[1]
// === /SERIES ===

// === PLOTTING ===
barcolor(color = closeSeries > openSeries ? #006600 : #990000, title = "Bar Colours")
// channel outline
closePlot   = plot(closeSeries, title = "Close Line", color = #009900, linewidth = 2, style = line, transp = 90)
openPlot    = plot(openSeries, title = "Open Line", color = #CC0000, linewidth = 2, style = line, transp = 90)
// channel fill
closePlotU  = plot(trendState ? closeSeries : na, transp = 100, editable = false)
openPlotU   = plot(trendState ? openSeries : na, transp = 100, editable = false)
closePlotD  = plot(trendState ? na : closeSeries, transp = 100, editable = false)
openPlotD   = plot(trendState ? na : openSeries, transp = 100, editable = false)
fill(openPlotU, closePlotU, title = "Up Trend Fill", color = #009900, transp = 40)
fill(openPlotD, closePlotD, title = "Down Trend Fill", color = #CC0000, transp = 40)
// === /PLOTTING ===

// === STRATEGY ===
// conditions
longCond    = crossover(closeSeries, openSeries)
shortCond   = crossunder(closeSeries, openSeries)
// entries and base exit
strategy.entry("long", strategy.long, when = longCond)
strategy.entry("short", strategy.short, when = shortCond)
// if we're using the trailing stop
if (useStop)
    strategy.exit("XL", from_entry = "long", trail_points = slPoints, trail_offset = slOffset)
    strategy.exit("XS", from_entry = "short", trail_points = slPoints, trail_offset = slOffset)
// not sure needed, but just incase..
strategy.exit("XL", from_entry = "long", when = shortCond)
strategy.exit("XS", from_entry = "short", when = longCond)
// === /STRATEGY ===
```

# Profitable?

- dual moving xmr-btc bittex 5m 12-84
- dual moving zec-btc bittex 5m 12-84
- ETH/BTC bittrex macd 13-6-4: 0.4% profit in 7 days, 100 btc, price range 0.005 btc, slightly bear/sideways

# Thoughts on configuration

- The config for the workers for the db is loaded by the workers from a json file. The config for apis for the workers comes from params sent to the task. This is why redis must be encrypted. It also allows for multi-tenant.
- The frontend and the workers must use the same database